# Predition Result on dev_tcn data by SMT

In [1]:
import random 
import pandas as pd
import numpy as np
from nltk.translate.ibm1 import IBMModel1
from nltk.translate.api import AlignedSent
import dill as pickle
import sacrebleu

#### PART 1 - ATTEMPT TO TRAIN MODEL BY DEV DATASET

In [2]:
dev_tcn = pd.read_csv('./preprocessed/cleaned_dev_tcn.csv')
dev_en = pd.read_csv('./preprocessed/cleaned_dev_en.csv')

In [3]:
dev_tcn.fillna(value='',inplace=True)
dev_en.fillna(value='',inplace=True)

In [4]:
dev_tcn_sents_all = dev_tcn["product_title_processed"].apply(lambda x: x.split())
dev_en_sents_all = dev_en["translation_output"].apply(lambda x: x.split())

In [5]:
max_count=1000
dev_en_sents = dev_en_sents_all[:max_count]
dev_tcn_sents = dev_tcn_sents_all[:max_count]

In [6]:
print("Size of english sentences: ", len(dev_tcn_sents_all ))
print("Size of french sentences: ", len(dev_en_sents_all))

Size of english sentences:  1000
Size of french sentences:  1000


In [7]:
dev_en_sents

0      [oppo, phone, case, soft, rabbit, silicone, case]
1         [soft, coating, car, wax, strong, water, watt]
2               [low, sugar, mango, dry, be, the, royal]
3      [the, culture, japan, imported, round, top, sp...
4      [hello, kitty, sandals, shoes, white, red, chi...
                             ...                        
995    [hippored, torn, fun, unique, style, straight,...
996    [kids, set, table, bay, thin, long, sleeve, ho...
997    [longchamp, le, pliage, neo, high, density, ny...
998      [ifairies, opening, adjustable, ring, ifairies]
999    [polarstar, women, sweat, quick, dry, t, shirt...
Name: translation_output, Length: 1000, dtype: object

In [8]:
dev_tcn_sents

0            [手机, 壳, 软壳, 挂绳, 壳, 大眼, 兔, 硅胶, 壳]
1                           [鍍膜車, 蠟, 強力, 撥水型]
2                            [低糖, 芒果乾, 臻, 御行]
3                            [小徑, 文化, 日本, 進口]
4                      [凱蒂貓, 涼鞋, 童鞋, 白紅色, 小童]
                        ...                  
995              [撕破, 乐趣, 独特, 风格, 中直, 筒, 牛仔裤]
996    [兒童, 套, 裝台, 灣, 製, 薄, 長, 袖, 居家, 套裝, 魔法]
997                       [高密, 尼龍, 後, 背包, 中型]
998                           [開口, 可, 調節, 戒指]
999                         [女排, 汗, 快干, 恤, 黑]
Name: product_title_processed, Length: 1000, dtype: object

In [9]:
aligned_text = []
for i in range(len(dev_en_sents)):
    al_sent2 = AlignedSent(dev_tcn_sents[i],dev_en_sents[i])
    aligned_text.append(al_sent2)
print("Training smt model")
ibm_model_ = IBMModel1(aligned_text,5)
print("Training complete")

Training smt model
Training complete


In [10]:
n_random = random.randint(0,max_count)
dev_tcn_sent = dev_tcn_sents_all[n_random]
dev_en_sent_actual_tr = dev_en_sents_all[n_random]
dev_tr_sent = []
for w in dev_tcn_sent:
    probs = ibm_model_.translation_table[w]
    if(len(probs)==0):
        continue
    sorted_words = sorted([(k,v) for k, v in probs.items()],key=lambda x: x[1], reverse=True)
    top_word = sorted_words[1][0]
    if top_word is not None:
        dev_tr_sent.append(top_word)
print("chinese sentence: ", " ".join(dev_tcn_sent))
print("Translated english sentence: ", " ".join(dev_tr_sent))
print("Original translation: ", " ".join(dev_en_sent_actual_tr))

chinese sentence:  遮陽 帽後開 衩 蝴蝶 結 草帽 防曬 帽子
Translated english sentence:  hat hat hat bowknot hat hat sunscreen hat
Original translation:  ifairies hat back slit bow straw hat sun hat


In [11]:
def translation(sentence):
    tr_sent = []
    for w in sentence:
        probs = ibm_model_.translation_table[w]
        if(len(probs)==0):
            continue
        sorted_words = sorted([(k,v) for k, v in probs.items()],key=lambda x: x[1], reverse=True)
        top_word = sorted_words[1][0]
        if top_word is not None:
            tr_sent.append(top_word)
    tr_sentence = []
    for i in tr_sent:
        tr_sentence.append(i)
    return " ".join(tr_sentence)

TEST SET TRANSLATION OUTPUT

In [12]:
test_set = pd.read_csv('./preprocessed/cleaned_test_tcn.csv').drop('split', axis=1)

In [13]:
test_set

,text,product_title_clean,product_title_processed
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊
2,粉晶六角柱純銀項鍊,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用
4,燈專屬優惠 *4盒,燈專屬優惠盒,燈 專屬 優惠盒
...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計


In [14]:
test_set.fillna(value='',inplace=True)

In [15]:
test_tcn_sents_all = test_set["product_title_processed"].apply(lambda x: x.split())


In [16]:
max_count=10000
test_tcn_sents = test_tcn_sents_all[:max_count]

In [17]:
test_set['prediction'] = test_tcn_sents.apply(lambda x: translation(x))

In [18]:
test_set

,text,product_title_clean,product_title_processed,prediction
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰,blend xotic gray
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊,daily millet aroma millet bracelet re bracelet
2,粉晶六角柱純銀項鍊,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊,bracelet
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用,washer momus
4,燈專屬優惠 *4盒,燈專屬優惠盒,燈 專屬 優惠盒,led
...,...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色,nylon bag kgh bondi block
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童,psv rescue
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電,sxn racing sxn
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計,desk mechanical motion eagle olfa dragon


In [19]:
test_set['prediction'] = test_set['prediction'].replace(r'^\s*$', np.NaN, regex=True) #removes empty strings or strings containing only spaces with NaN

In [20]:
test_set.dropna(inplace=True)
test_set

,text,product_title_clean,product_title_processed,prediction
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰,blend xotic gray
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊,daily millet aroma millet bracelet re bracelet
2,粉晶六角柱純銀項鍊,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊,bracelet
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用,washer momus
4,燈專屬優惠 *4盒,燈專屬優惠盒,燈 專屬 優惠盒,led
...,...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色,nylon bag kgh bondi block
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童,psv rescue
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電,sxn racing sxn
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計,desk mechanical motion eagle olfa dragon


In [21]:
test_set.to_csv('./output/0_SMT_test_output1.csv', index = False, encoding='utf_8_sig')

USING PARALLEL DEV SET TO EVALUATE BLEU SCORE

In [22]:
dev_tcn['prediction'] = dev_tcn_sents.apply(lambda x: translation(x))

In [23]:
dev_tcn['prediction'] = dev_tcn['prediction'].replace(r'^\s*$', np.NaN, regex=True) #removes empty strings or strings containing only spaces with NaN

In [24]:
tr_nan_indexes = dev_tcn.loc[pd.isna(dev_tcn).any(1), :].index #stores nan indexes for later dev_en follow to drop nan rows as well
tr_nan_indexes

Int64Index([260, 505, 568, 593, 597, 626], dtype='int64')

In [25]:
dev_tcn.dropna(inplace=True)
dev_tcn.drop('split', axis=1, inplace=True)
dev_tcn

,text,product_title_clean,product_title_processed,prediction
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,手机壳软壳挂绳壳大眼兔硅胶壳,手机 壳 软壳 挂绳 壳 大眼 兔 硅胶 壳,asus play oppo oppo play oppo oppo oppo play
1,SOFT 99 鍍膜車蠟(強力撥水型),鍍膜車蠟強力撥水型,鍍膜車 蠟 強力 撥水型,wax wax watt wax
2,低糖芒果乾 250g 臻御行,低糖芒果乾臻御行,低糖 芒果乾 臻 御行,be be be be
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,小徑文化日本進口,小徑 文化 日本 進口,craft craft fco craft
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,凱蒂貓涼鞋童鞋白紅色小童,凱蒂貓 涼鞋 童鞋 白紅色 小童,kitty flashing psv hello flashing
...,...,...,...,...
995,【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445,撕破乐趣独特风格中直筒牛仔裤,撕破 乐趣 独特 风格 中直 筒 牛仔裤,unique unique unique unique unique tail unique
996,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,兒童套裝台灣製薄長袖居家套裝魔法,兒童 套 裝台 灣 製 薄 長 袖 居家 套裝 魔法,children techniques table desk desk thin xxl k...
997,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),高密尼龍後背包中型,高密 尼龍 後 背包 中型,density nylon nylon bag density
998,iFairies 開口可調節戒指★ifairies【56472】【56472】,開口可調節戒指,開口 可 調節 戒指,opening komax ifairies opening


In [26]:
dev_tcn.to_csv('./output/0_SMT_dev_output1.csv', index=False, encoding='utf_8_sig')

In [27]:
dev_tcn = pd.read_csv('./output/0_SMT_dev_output1.csv')

In [28]:
dev_en.drop(tr_nan_indexes, inplace=True)

In [29]:
dev_en["translation_output"]

0              oppo phone case soft rabbit silicone case
1                 soft coating car wax strong water watt
2                       low sugar mango dry be the royal
3      the culture japan imported round top space cra...
4           hello kitty sandals shoes white red children
                             ...                        
995        hippored torn fun unique style straight jeans
996    kids set table bay thin long sleeve home suit ...
997    longchamp le pliage neo high density nylon bac...
998            ifairies opening adjustable ring ifairies
999        polarstar women sweat quick dry t shirt black
Name: translation_output, Length: 994, dtype: object

In [30]:
refs = [dev_en["translation_output"]]
sys = dev_tcn['prediction']
bleu = sacrebleu.corpus_bleu(sys, refs, lowercase=True)
print(bleu.precisions)

[31.922276197085356, 3.3752620545073375, 0.079155672823219, 0.017605633802816902]


#### \*\*\* PART 2 - ATTEMPT TO TRAIN MODEL BY MATCHED GOOGLE TRANSLATE DATASET\*\*\*

In [31]:
df = pd.read_csv('./preprocessed/train_matched_combined.csv')
df

,Original,Translated,Matched,Cosine_sim
0,耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳,Niji official men's football trousers new stan...,trousers,0.555
1,火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服,Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...,flat sheet blanket cotton,0.000
2,小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋,Small thumb shoes gold standard shell head whi...,fashion sports shoes women s shoes,0.784
3,居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 ...,Home Big Push Memory Cotton Pillow Care Cervic...,flat sheet blanket cotton,0.000
4,炫彩 細口手 沖 壺桃紅 藍 黃,Colorful thin mouthful pot pot red blue yellow,mini pot extra small pot taho pot,0.526
...,...,...,...,...
135995,噴霧 瓶雪寧 稀釋 瓶 噴霧 瓶 有 刻度 家用 塑料瓶 小噴壺 廚房 洗潔 精分 裝瓶,Spray bottle Snow Ning dilute bottle spray bot...,bottle,0.791
135996,現貨 三角 造型 烤盤 造型 盤餐 盤,Spot triangle styling baking pan styling dishes,hair styling twist,0.385
135997,歐美韓 爆款韓 名媛 英倫學院 風搖滾 龐克 仿舊 性感 百搭 宜興 紫砂 杯純 全手工 茶...,"Europe and the United States, Korean explosive...",tea cup,0.639
135998,新款 字母 月亮 元素 珍珠 耳釘 別 緻 的 一款 經典 時尚,New alphabet moon elements Pearl earrings shou...,fashion earrings,0.408


DROPPING NON-ENGLISH TRANSLATED ROWS (FAILED TRANSLATIONS BY GOOGLE API)

In [32]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [33]:
df.drop(df[~df['Translated'].apply(lambda x : isEnglish(x))].index, inplace = True)

In [34]:
df_clean = df.reset_index(drop=True)
df_clean

,Original,Translated,Matched,Cosine_sim
0,耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳,Niji official men's football trousers new stan...,trousers,0.555
1,火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服,Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...,flat sheet blanket cotton,0.000
2,小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋,Small thumb shoes gold standard shell head whi...,fashion sports shoes women s shoes,0.784
3,居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 ...,Home Big Push Memory Cotton Pillow Care Cervic...,flat sheet blanket cotton,0.000
4,炫彩 細口手 沖 壺桃紅 藍 黃,Colorful thin mouthful pot pot red blue yellow,mini pot extra small pot taho pot,0.526
...,...,...,...,...
130053,噴霧 瓶雪寧 稀釋 瓶 噴霧 瓶 有 刻度 家用 塑料瓶 小噴壺 廚房 洗潔 精分 裝瓶,Spray bottle Snow Ning dilute bottle spray bot...,bottle,0.791
130054,現貨 三角 造型 烤盤 造型 盤餐 盤,Spot triangle styling baking pan styling dishes,hair styling twist,0.385
130055,歐美韓 爆款韓 名媛 英倫學院 風搖滾 龐克 仿舊 性感 百搭 宜興 紫砂 杯純 全手工 茶...,"Europe and the United States, Korean explosive...",tea cup,0.639
130056,新款 字母 月亮 元素 珍珠 耳釘 別 緻 的 一款 經典 時尚,New alphabet moon elements Pearl earrings shou...,fashion earrings,0.408


In [35]:
train_tcn_sents_all = df_clean["Original"].apply(lambda x: x.split())
train_en_sents_all = df_clean["Translated"].apply(lambda x: x.split())

In [36]:
max_count=130058
train_en_sents = train_en_sents_all[:max_count]
train_tcn_sents = train_tcn_sents_all[:max_count]

In [37]:
print("Size of traditional chinese sentences: ", len(train_tcn_sents_all ))
print("Size of english sentences: ", len(train_en_sents_all))

Size of traditional chinese sentences:  130058
Size of english sentences:  130058


In [38]:
aligned_text = []
for i in range(len(train_en_sents)):
    al_sent2 = AlignedSent(train_tcn_sents[i],train_en_sents[i])
    aligned_text.append(al_sent2)
print("Training smt model")
ibm_model_ = IBMModel1(aligned_text,5)
print("Training complete")

Training smt model
Training complete


In [39]:
n_random = random.randint(0,max_count)
train_tcn_sent = train_tcn_sents_all[n_random]
train_en_sent_actual_tr = train_en_sents_all[n_random]
train_tr_sent = []
for w in train_tcn_sent:
    probs = ibm_model_.translation_table[w]
    if(len(probs)==0):
        continue
    sorted_words = sorted([(k,v) for k, v in probs.items()],key=lambda x: x[1], reverse=True)
    top_word = sorted_words[1][0]
    if top_word is not None:
        train_tr_sent.append(top_word)
print("chinese sentence: ", " ".join(train_tcn_sent))
print("Translated english sentence: ", " ".join(train_tr_sent))
print("Original translation: ", " ".join(train_en_sent_actual_tr))

chinese sentence:  不沾鍋
Translated english sentence:  Xiangyuan
Original translation:  Do not touch the pot


In [40]:
test_set = pd.read_csv('./preprocessed/cleaned_test_tcn.csv')

In [41]:
test_set

,text,split,product_title_clean,product_title_processed
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,public,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊
2,粉晶六角柱純銀項鍊,public,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,public,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用
4,燈專屬優惠 *4盒,public,燈專屬優惠盒,燈 專屬 優惠盒
...,...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,private,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,private,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,private,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",private,達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計


In [42]:
test_set.fillna(value='',inplace=True)

In [43]:
test_tcn_sents_all = test_set["product_title_processed"].apply(lambda x: x.split())


In [44]:
max_count=10000
test_tcn_sents = test_tcn_sents_all[:max_count]

In [45]:
test_set['prediction'] = test_tcn_sents.apply(lambda x: translation(x))

In [46]:
test_set

,text,split,product_title_clean,product_title_processed,prediction
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,public,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰,Yujing Wool Warm exoction
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊,Crotted Crystal natural Crystal anti-scared wa...
2,粉晶六角柱純銀項鍊,public,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊,cherish hexagon Clay-plated chain
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,public,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用,search-free Double-sided outdoor Dedicated
4,燈專屬優惠 *4盒,public,燈專屬優惠盒,燈 專屬 優惠盒,lights Exclusive
...,...,...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,private,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色,"after Backpack Shengdong tensile brand, white"
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,private,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童,Barli freezed Tianyu
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,private,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電,Disneyvi Action Power Terui Duck Disneyvi Duck...
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",private,達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計,Saller-shaped system figure lead-free Pen Bala...


In [47]:
test_set['prediction'] = test_set['prediction'].replace(r'^\s*$', np.NaN, regex=True) #removes empty strings or strings containing only spaces with NaN

In [48]:
test_set.dropna(inplace=True)
test_set.drop('split', axis=1, inplace=True)
test_set

,text,product_title_clean,product_title_processed,prediction
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰,Yujing Wool Warm exoction
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊,Crotted Crystal natural Crystal anti-scared wa...
2,粉晶六角柱純銀項鍊,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊,cherish hexagon Clay-plated chain
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用,search-free Double-sided outdoor Dedicated
4,燈專屬優惠 *4盒,燈專屬優惠盒,燈 專屬 優惠盒,lights Exclusive
...,...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色,"after Backpack Shengdong tensile brand, white"
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童,Barli freezed Tianyu
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電,Disneyvi Action Power Terui Duck Disneyvi Duck...
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計,Saller-shaped system figure lead-free Pen Bala...


In [49]:
test_set.to_csv('./output/0_SMT_test_output2.csv', index = False, encoding='utf_8_sig')

USING PARALLEL DEV SET TO EVALUATE BLEU SCORE

In [50]:
dev_tcn = pd.read_csv('./preprocessed/cleaned_dev_tcn.csv')
dev_en = pd.read_csv('./preprocessed/cleaned_dev_en.csv')

In [51]:
dev_tcn.fillna(value='',inplace=True)
dev_en.fillna(value='',inplace=True)

In [52]:
dev_tcn['prediction'] = dev_tcn_sents.apply(lambda x: translation(x))

In [53]:
dev_tcn

,text,split,product_title_clean,product_title_processed,prediction
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,private,手机壳软壳挂绳壳大眼兔硅胶壳,手机 壳 软壳 挂绳 壳 大眼 兔 硅胶 壳,"two-band rope, Pench Rabbit Pai"
1,SOFT 99 鍍膜車蠟(強力撥水型),private,鍍膜車蠟強力撥水型,鍍膜車 蠟 強力 撥水型,Wax Strong
2,低糖芒果乾 250g 臻御行,private,低糖芒果乾臻御行,低糖 芒果乾 臻 御行,"containing mango ore,"
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,private,小徑文化日本進口,小徑 文化 日本 進口,Germus Culture Japan's import
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,private,凱蒂貓涼鞋童鞋白紅色小童,凱蒂貓 涼鞋 童鞋 白紅色 小童,Kati Sandals freezed Teeth superhand
...,...,...,...,...,...
995,【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445,private,撕破乐趣独特风格中直筒牛仔裤,撕破 乐趣 独特 风格 中直 筒 牛仔裤,torn Little low-cartoise yager
996,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,private,兒童套裝台灣製薄長袖居家套裝魔法,兒童 套 裝台 灣 製 薄 長 袖 居家 套裝 魔法,"Child Anti-scratch-raised paved Bay, system ra..."
997,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),private,高密尼龍後背包中型,高密 尼龍 後 背包 中型,Gaomiff Nylon after Backpack medium-sized
998,iFairies 開口可調節戒指★ifairies【56472】【56472】,private,開口可調節戒指,開口 可 調節 戒指,laughing be adjustment ring


In [54]:
dev_tcn['prediction'] = dev_tcn['prediction'].replace(r'^\s*$', np.NaN, regex=True) #removes empty strings or strings containing only spaces with NaN

In [55]:
tr_nan_indexes = dev_tcn.loc[pd.isna(dev_tcn).any(1), :].index #stores nan indexes for later dev_en follow to drop nan rows as well
tr_nan_indexes

Int64Index([213, 260, 458, 500, 505, 568, 593, 597, 626, 747, 788, 814, 850], dtype='int64')

In [56]:
dev_tcn.dropna(inplace=True)
dev_tcn.drop('split', axis=1, inplace=True)
dev_tcn

,text,product_title_clean,product_title_processed,prediction
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,手机壳软壳挂绳壳大眼兔硅胶壳,手机 壳 软壳 挂绳 壳 大眼 兔 硅胶 壳,"two-band rope, Pench Rabbit Pai"
1,SOFT 99 鍍膜車蠟(強力撥水型),鍍膜車蠟強力撥水型,鍍膜車 蠟 強力 撥水型,Wax Strong
2,低糖芒果乾 250g 臻御行,低糖芒果乾臻御行,低糖 芒果乾 臻 御行,"containing mango ore,"
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,小徑文化日本進口,小徑 文化 日本 進口,Germus Culture Japan's import
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,凱蒂貓涼鞋童鞋白紅色小童,凱蒂貓 涼鞋 童鞋 白紅色 小童,Kati Sandals freezed Teeth superhand
...,...,...,...,...
995,【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445,撕破乐趣独特风格中直筒牛仔裤,撕破 乐趣 独特 风格 中直 筒 牛仔裤,torn Little low-cartoise yager
996,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,兒童套裝台灣製薄長袖居家套裝魔法,兒童 套 裝台 灣 製 薄 長 袖 居家 套裝 魔法,"Child Anti-scratch-raised paved Bay, system ra..."
997,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),高密尼龍後背包中型,高密 尼龍 後 背包 中型,Gaomiff Nylon after Backpack medium-sized
998,iFairies 開口可調節戒指★ifairies【56472】【56472】,開口可調節戒指,開口 可 調節 戒指,laughing be adjustment ring


In [57]:
dev_tcn.to_csv('./output/0_SMT_dev_output2.csv', index=False, encoding='utf_8_sig')

In [58]:
dev_tcn = pd.read_csv('./output/0_SMT_dev_output2.csv')

In [59]:
dev_en.drop(tr_nan_indexes, inplace=True)

In [60]:
dev_en["translation_output"]

0              oppo phone case soft rabbit silicone case
1                 soft coating car wax strong water watt
2                       low sugar mango dry be the royal
3      the culture japan imported round top space cra...
4           hello kitty sandals shoes white red children
                             ...                        
995        hippored torn fun unique style straight jeans
996    kids set table bay thin long sleeve home suit ...
997    longchamp le pliage neo high density nylon bac...
998            ifairies opening adjustable ring ifairies
999        polarstar women sweat quick dry t shirt black
Name: translation_output, Length: 987, dtype: object

In [61]:
refs = [dev_en["translation_output"]]
sys = dev_tcn['prediction']
bleu = sacrebleu.corpus_bleu(sys, refs, lowercase=True)
print(bleu.precisions)

[24.01134953005852, 5.180567497850387, 1.2965964343598055, 0.3523608174770965]
